In [10]:
from coinbase.wallet.client import Client
import pandas as pd
import requests

In [11]:
api_key = "abRJTjZ5f940NqDm"
api_secret = "iZUizAftYAExBKIFktworjDksAWajsHU"
client = Client(api_key, api_secret)

In [15]:

url = "https://api.exchange.coinbase.com/products/LINK-USD/candles"
response = requests.get(url)
kline_data  = eval(response.text)
headers = ["timestamp", "low", "high", "open", "close", "volume"]
df = pd.DataFrame(data=kline_data, columns=headers)
df



,timestamp,low,high,open,close,volume
0,1670201820,7.480,7.490,7.480,7.490,388.02
1,1670201760,7.480,7.491,7.490,7.480,1001.64
2,1670201700,7.487,7.492,7.488,7.490,1465.34
3,1670201640,7.482,7.491,7.483,7.490,1836.29
4,1670201580,7.482,7.488,7.486,7.483,1748.99
...,...,...,...,...,...,...
295,1670184120,7.381,7.383,7.382,7.383,408.21
296,1670184060,7.380,7.382,7.380,7.382,519.96
297,1670184000,7.372,7.380,7.372,7.380,2350.12
298,1670183940,7.371,7.373,7.371,7.372,1192.86
